<a target="_blank" href="https://colab.research.google.com/github/gox6/colab-demos/blob/main/rags/evaluate-rags-rigorously-or-perish.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [4]:
# Installing Python packages & hiding
!pip install langchain ragas xmltodict &> /dev/null

In [5]:
# Managing secrets
# - If using Colab please use Colab Secrets
# - If running outside Colab please provide secrets as ENV VARS

import os

COLAB = os.getenv("COLAB_RELEASE_TAG") is not None

if COLAB:
  from google.colab import userdata
  OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
  os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
else:
  OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')



In [6]:
from langchain.document_loaders import PubMedLoader

loader = PubMedLoader("liver", load_max_docs=10)
documents = loader.load()

Too Many Requests, waiting for 0.20 seconds...
Too Many Requests, waiting for 0.40 seconds...
Too Many Requests, waiting for 0.80 seconds...
Too Many Requests, waiting for 1.60 seconds...


In [9]:
documents

[Document(page_content="INTRODUCTION: Uterine fibroids, the most prevalent benign tumors among reproductive-age women, pose treatment challenges that range from surgical interventions to medical therapies for symptom control. Progestins and estroprogestins effectively manage uterine bleeding by suppressing dysfunctional endometrium over fibroids. While GnRH agonists represent a crucial milestone in symptom treatment, their prolonged use results in menopausal-like symptoms and irreversible bone mineral density loss. Advancements in understanding fibroid pathophysiology have prompted the exploration of new compounds to overcome current therapy limitations.\nAREAS COVERED: This manuscript offers an updated overview of investigational drugs for symptomatic uterine fibroids.\nEXPERT OPINION: Despite ulipristal acetate's well-established efficacy as a selective progesterone receptor modulator (SPRM) in fibroid treatment, its prescription has declined due to the rare but severe risk of liver 

In [10]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# documents = load your documents

# generator with openai models
generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
critic_llm = ChatOpenAI(model="gpt-4")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

# Change resulting question type distribution
distributions = {
    simple: 0,
    multi_context: 1,
    reasoning: 0
}

# use generator.generate_with_llamaindex_docs if you use llama-index as document loader
testset = generator.generate_with_langchain_docs(documents, 1, distributions)
testset.to_pandas()

embedding nodes:   0%|          | 0/20 [00:00<?, ?it/s]

Generating:   0%|          | 0/1 [00:00<?, ?it/s]

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What are the effects of PSO supplementation on...,"[['Muribaculaceae', 'Blautia']\nPunicic acid (...",PSO supplementation reduces hepatic lipid accu...,multi_context,"[{'uid': '38618675', 'Title': {'i': 'via', '#t...",True
